# 加载数据集
路径:C:\Users\Administrator\Desktop\college\junior2\工业大数据\实验\lab1\data\all_data_ct_10M.h5

In [1]:
data_path = 'C:\\Users\\Administrator\\Desktop\\college\\junior2\\工业大数据\\实验\\lab1\\data\\all_data_ct_10M.h5'

from h5Reader.readHdf5 import readH5

data,idx = readH5(data_path,datasets=['data','idx'])

# 获取数据集原始形状
8928*10000*5

In [6]:
# 原始数据集
data_shape = data.shape
num_timePoints = data_shape[0]
num_districts = data_shape[1]
num_business = data_shape[2]

print("时间点数量:%d,小区数量:%d,流量业务类型数量:%d" %(num_timePoints,num_districts,num_business))

时间点数量:8928,小区数量:10000,流量业务类型数量:5


# 划分数据集
按0.7的比重划分，总计8928个时间点，也即8928*0.8=6249.59≈6250条数据

In [10]:
import numpy as np
seperator_index = int(np.round((num_timePoints*0.7)))
#print(seperator_index)

train_set = data[0:seperator_index,:,:]
test_set = data[seperator_index:num_timePoints,:,:]
# print(train_set.shape,test_set.shape)

(6250, 10000, 5) (2678, 10000, 5)


# 构造输入输出序列
CNN希望接收的参数格式: ( batch_size, time_steps, features )
放到这个例子中就是( 样本数量, 时间步长, 特征数量 )
假设我们的时间序列的滑动窗口大小为4,构建输入输出序列

In [13]:
def restructure(set, win_size):
    X_seq,Y_seq = [],[]
    shape = set.shape
    tps = shape[0]
    districts = shape[1]
    business = shape[2]
    
    for t in range(win_size,tps):
        X_seq.append(set[t-win_size+1:t+1,:,:])
        Y_seq.append(set[t+1,:,:])
    
    X_seq = np.array(X_seq).reshape(-1,win_size,districts,business)
    Y_seq = np.array(Y_seq).reshape(-1,districts,business)
    
    return X_seq,Y_seq

window_size = 4
X_train,Y_train = restructure(train_set,window_size)
X_test,Y_test = restructure(test_set,window_size)

IndexError: index 6250 is out of bounds for axis 0 with size 6250

此时
X=(train_tps-window_size,window_size,train_districts,train_business)
Y=(train_tps-window_size,train_districts,train_business)
# 构建CNN网络模型
卷积层(32卷积核大小为3)+最大池化层+全连接层+输出层(线性激活函数)

In [12]:
import tensorflow as tf
#print(tf.__version__) # tensorflow版本在2.x才能使用tf.keras

model = tf.keras.Sequential()

# 第一个卷积层，使用32个卷积核，大小为3
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu',input_shape = (window_size,X_train.shape[3])))

# 全局最大池化层
model.add(tf.keras.layers.GlobalMaxPooling1D())

# 全连接层
model.add(tf.keras.layers.Dense(64, activation='relu'))

# 输出层，使用线性激活函数进行回归预测
model.add(tf.keras.layers.Dense(1, activation='linear'))

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')

2.14.0


# 训练模型
训练+评估性能

In [ ]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# 评估模型
loss = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test loss: {loss}")

# 模型预测

In [ ]:
# 使用模型进行预测
predictions = model.predict(X_test)